# 3D Rotation
3D Rotation can be represented in different ways as discussed in last week's lectures.
These include:
1.	Rotation matrices
2.  Three angle representation
3.	Two vector representation
4.	Rotation about an eigen vector
5.	Unit quaternion
6.	Matrix exponential

We explored the `rotation matrices` approach in last weeks lab exercise.

This lab is designed for you to implement rotation matrices in each of those different methods.

In [1]:
# import required libraries
import numpy as np
import matplotlib.pyplot as plt
# To render plots inline
%matplotlib inline

# For unit tests
from tester import LabTester
lab_tester = LabTester()

In [2]:
def rotation_three_angle(alpha:float, gamma:float, theta:float, p:np.ndarray) -> np.ndarray:
    """
    Input:
        alpha, gamma, theta - Euler angles of rotation about the axes (x, y, x)
        p                   - The vector to be transformed
    Output: resulting coordinates after appyling the rotation
    """
    # Your code here - Use Euler angles method => (x,y,x)

    def rotation_x(angle: float)-> np.ndarray:
        angle = np.radians(angle)

        R_x = np.array([
            [1, 0, 0],
            [0, np.cos(angle), -np.sin(angle)],
            [0, np.sin(angle), np.cos(angle)]
        ])

        return R_x

    def rotation_y(angle: float)-> np.ndarray:
        angle = np.radians(angle)

        R_y = np.array([
            [np.cos(angle), 0, np.sin(angle)],
            [0, 1, 0],
            [-np.sin(angle), 0, np.cos(angle)]
        ])

        return R_y

    p = rotation_x(alpha) @ rotation_y(gamma) @ rotation_x(theta) @ p
    return p


In [3]:
# Run test cases
lab_tester.test_rotation_three_angle(rotation_three_angle)


=== Testing Three Angle Rotation Implementation ===
✅ No Rotation
✅ 45 Degree Rotation
✅ Full Rotation
✅ Negative 45 Degree Rotation


In [4]:
def rotation_two_vector(a_v: np.ndarray, o_v: np.ndarray, p:np.ndarray) -> np.ndarray:
    """
    Input:
        a_v     - Approach vector
        o_v     - Orientation vector
        p       - The vector to be transformed
    Output: resulting coordinates after appyling the rotation
    """
    # Your code here
    a_v = a_v / np.linalg.norm(a_v)
    o_v = o_v / np.linalg.norm(o_v)

    orthogonal_vector = np.cross(o_v, a_v)
    o_v = np.cross(a_v, orthogonal_vector)

    if(np.linalg.norm(orthogonal_vector) < 1e-6):
        raise ValueError("Gimbal Lock Occurred: Approach vector and Orientation Vector are parallel or anti parallel")

    # Normalization
    orthogonal_vector = orthogonal_vector / np.linalg.norm(orthogonal_vector)
    o_v = o_v / np.linalg.norm(o_v)
    a_v = a_v / np.linalg.norm(a_v)
    # Rotational Matrix
    result = np.array([orthogonal_vector, o_v, a_v])
    R = result.T
    return R @ p

In [5]:
lab_tester.test_rotation_two_vector(rotation_two_vector)


=== Testing Two Vector Rotation Implementation ===
✅ X-Axis Rotation
✅ Y-Axis Rotation
✅ Z-Axis Rotation
✅ 45-degree Z Rotation


In [6]:
def rotation_eigen_vector(neta: np.ndarray, theta:float, p:np.ndarray) -> np.ndarray:
    """
    Input:
        neta    - Eigen vector with eigen value 1
        theta   - Angle of rotation about neta
        p       - The vector to be transformed
    Output: resulting coordinates after appyling the rotation
    """

    neta = neta / np.linalg.norm(neta)
    theta = np.radians(theta)

    s = np.cos(theta/2)
    omega = neta * np.sin(theta/2)

    p1 = (np.cross(omega, p)) * 2 * s
    p2 = 2*(np.cross(omega, np.cross(omega, p)))

    return p + p1 + p2

In [7]:
lab_tester.test_rotation_eigen_vector(rotation_eigen_vector)


=== Testing Eigen Vector Rotation Implementation ===
✅ No Rotation
✅ 180-degree Z Rotation
✅ 90-degree X Rotation
✅ 45-degree Y Rotation
✅ Negative 90-degree Z Rotation


In [8]:
def rotation_unit_quaternion(q: np.ndarray, p: np.ndarray) -> np.ndarray:
    # Your code here - q: Unit quaternion vector [Real, i, j, k]

    q = q / np.linalg.norm(q)

    real, i, j, k = q

    q_inv = np.array([real, -i, -j, -k])
    p = np.array([0, p[0], p[1], p[2]])

    def mul_quaternion(q1: np.ndarray, q2: np.ndarray):
      s1, v1 = q1[0], q1[1:]
      s2, v2 = q2[0], q2[1:]

      s = s1*s2 - np.dot(v1, v2)
      v = s1*v2 + s2*v1 + np.cross(v1, v2)
      return np.array([s, v[0], v[1], v[2]])

    first_rotation = mul_quaternion(q, p)
    second_rotation = mul_quaternion(first_rotation, q_inv)

    return second_rotation[1:]

In [9]:
lab_tester.test_rotation_unit_quaternion(rotation_unit_quaternion)


=== Testing Unit Quaternion Rotation Implementation ===
✅ No Rotation
✅ 90-degree X Rotation
✅ 180-degree Y Rotation
✅ 90-degree Z Rotation
✅ Negative 180-degree X Rotation


In [10]:
from scipy import linalg

In [11]:
def rotation_matrix_exp(w_x : np.ndarray, theta: float, p: np.ndarray) -> np.ndarray:
    x, y, z = w_x / np.linalg.norm(w_x)

    m_skew = np.array([
        [0, -z, y],
        [z, 0, -x],
        [-y, x, 0]
    ])

    R = linalg.expm(np.radians(theta) * m_skew)

    return R @ p
rotation_matrix_exp([1, 0, 0], 0.3, [1, 1, 1]).round(1)

array([1., 1., 1.])

In [12]:
lab_tester.test_rotation_matrix_exp(rotation_matrix_exp)


=== Testing Matrix Exponential Rotation Implementation ===
✅ No Rotation
✅ 90-degree X-axis Rotation
✅ 90-degree Y-axis Rotation
✅ 90-degree Z-axis Rotation
✅ 180-degree X-axis Rotation
✅ 180-degree Y-axis Rotation
✅ 180-degree Z-axis Rotation
✅ 45-degree X-axis Rotation
